In [1]:
# Importamos todas las librerías necesarias para utilizar Spark Streaming con Python desde
# JUpyter Notebook

#import findspark
#findspark.init()
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
!pip3 install numpy
import numpy

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml import Pipeline

In [2]:
# Importamos todas las librerías que vamos a necesitar para ejecutar los Scripts con Spark
import sys
import re
import nltk
import ast
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import spacy

englishST = open('/home/user/englishST.txt', 'r', encoding = 'latin-1')
stop = englishST.read().splitlines()
englishST.close()

sb_english = SnowballStemmer('english')

2022-03-16 20:25:17.996348: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-03-16 20:25:17.996370: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Escribimos los scripts que se utilizarán con las transformaciones de Spark

# 1º Eliminación de las "Regular Expressions"
def regex(text):
    new_text = re.sub("\W+", " ", text)#elimina puntos comas,etc
    new_text = re.sub("\d+", " ", new_text).strip()#elimina numeros
    return new_text
    
   
# 2º Ponemos todo el contenido en minúscula
def lower_case(text): 
    new_text = linea.lower()
    return new_text


# 3º Tokenización de las oraciones
def tokenizer(text):
    new_text = word_tokenize(text, language="english")
    return new_text

# 4º Eliminamos las StopWords
def stop_words(text):
    text = text.splitlines()
    for words in text:
        words = ast.literal_eval(words)
        new_sentence = []

        for word in words:
            if word not in stop:
                new_sentence.append(word)

        if new_sentence:
            return new_sentence
        else:
            return []


def filter(text):

    if text != []:
        return text

    
# 5º Stemming
def stemmer(text):
    # Obtenemos el lemma por cada token
    text = text.splitlines()
    for words in text:
        sentence = []
        words = ast.literal_eval(words)
        for word in words:
            sentence.append(sb_english.stem(word))

    return sentence


# 6º Lemmatización
def lemmatization(text):
    text = text.splitlines()
    for words in text:
        sentence = []
        words = ast.literal_eval(words)

    # obtenemos los lemmas
    doc = nlp(" ".join(words))
    for word in doc:
        sentence.append(word.lemma_)

    return sentence

# 7º Volvemos a formar la oración a partir de los tokens
def join_all(text):

    text = text.splitlines()
    for words in text:
        words = ast.literal_eval(words)	
        new_text = " ".join(words)
        
    return new_text

In [2]:
start = time.time()
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param, Params, TypeConverters
from pyspark.ml.util import DefaultParamsReadable, DefaultParamsWritable
from pyspark.sql import DataFrame
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
from SentiLib.text import predict_emotions_text
from pyspark.sql.functions import udf

class CustomTransformer(Transformer, HasInputCol, HasOutputCol, DefaultParamsReadable, DefaultParamsWritable):
    input_col = Param(Params._dummy(), "input_col", "input column name.", typeConverter=TypeConverters.toString)
    output_col = Param(Params._dummy(), "output_col", "output column name.", typeConverter=TypeConverters.toString)
  
    @keyword_only
    def __init__(self, input_col: str = "input", output_col: str = "output"):
        super(CustomTransformer, self).__init__()
        self._setDefault(input_col=None, output_col=None)
        kwargs = self._input_kwargs
        self.set_params(**kwargs)
    
    @keyword_only
    def set_params(self, input_col: str = "input", output_col: str = "output"):
        kwargs = self._input_kwargs
        self._set(**kwargs)
    
    def get_input_col(self):
        return self.getOrDefault(self.input_col)
  
    def get_output_col(self):
        return self.getOrDefault(self.output_col)
  
    def _transform(self, df: DataFrame):
        input_col = self.get_input_col()
        output_col = self.get_output_col()
        # The custom action: concatenate the integer form of the doubles from the Vector
        transform_udf = F.udf(lambda x: str(predict_emotions_text(x)), StringType())
        return df.withColumn(output_col, transform_udf(input_col))
end = time.time()
print (end - start)

2022-03-28 16:20:12.839407: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-03-28 16:20:12.839434: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


4.945544242858887


In [3]:
# Creamos los pipelines con Spark NLP
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.ml import Pipeline


#spark = sparknlp.start()

spark = SparkSession.builder \
    .appName("Spark NLP")\
    .master("local")\
    .config("spark.driver.memory","16G")\
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.kryoserializer.buffer.max", "2000M")\
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.4.2")\
    .getOrCreate()

# Tomamos de la carpeta "Datos" los archivos txt que van entrando con las diferentes frases
# Estos archivos se toman de uno en uno.
text = spark.read \
.format("text") \
.text("file:///home/user/Datos/texto.txt")


print(sparknlp.version())
print(spark.version)
Document = DocumentAssembler().setInputCol('value').setOutputCol('document').setCleanupMode('shrink')
Sentence = SentenceDetector().setInputCols('document').setOutputCol('sentence')
Sentence.setExplodeSentences(True)
Tokenizer = RegexTokenizer().setInputCols('sentence').setOutputCol('token')
Normalizer = Normalizer().setInputCols(['token']).setOutputCol('normalize').setLowercase(True).setCleanupPatterns([])
Stemmer = Stemmer().setInputCols('normalize').setOutputCol('stemming')
TokenAssembler = TokenAssembler().setInputCols("sentence", "stemming").setOutputCol("cleanText")
Finisher = Finisher().setInputCols('cleanText').setOutputCols('output').setOutputAsArray(False)
custom_transformer = CustomTransformer(input_col="output", output_col="sentiment")

pipeline = Pipeline().setStages([
    Document,
    Sentence,
    Tokenizer,
    Normalizer,
    Stemmer,
    TokenAssembler,
    Finisher,
    custom_transformer
])

model = pipeline.fit(text)
result = model.transform(text)

2022-03-28 16:20:31,333 WARN util.Utils: Your hostname, fabio resolves to a loopback address: 127.0.1.1; using 192.168.100.23 instead (on interface eno1)
2022-03-28 16:20:31,334 WARN util.Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/spark-3.2.1-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fabio/.ivy2/cache
The jars for the packages stored in: /home/fabio/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-6cfdab31-ff08-4c94-a6b7-485ce8332aeb;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;3.4.2 in central
	found com.typesafe#config;1.4.1 in central
	found org.rocksdb#rocksdbjni;6.5.3 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.603 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#lombok

3.4.1
3.2.1


In [4]:
# Transformaciones y acciones
start = time.time()
result.show()
end = time.time()
print (end - start)

2022-03-28 16:20:50.989792: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ros/noetic/lib
2022-03-28 16:20:50.989810: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


+--------------------+--------------------+--------------------+
|               value|              output|           sentiment|
+--------------------+--------------------+--------------------+
|THEY ENJOY IT WHE...|thei enjoi it whe...|{'anger': 0, 'ant...|
|THE VIEWPOINT OVE...|the viewpoint ove...|{'anger': 0, 'ant...|
|THE DIAGNOSIS WAS...|the diagnosi wa d...|{'anger': 0, 'ant...|
|RIGHT NOW MAY NOT...|right now mai not...|{'anger': 0, 'ant...|
|THE CARPETCLEAN A...|the carpetclean a...|{'anger': 0, 'ant...|
|AGRICULTURAL PROD...|agricultur produc...|{'anger': 0, 'ant...|
|NO MANUFACTURER H...|no manufactur ha ...|{'anger': 1, 'ant...|
|KINDERGART AN CHI...|kindergart an chi...|{'anger': 0, 'ant...|
|DON'T ASK ME TO C...|don't ask me to c...|{'anger': 1, 'ant...|
|THE NEAREST SYNAG...|the nearest synag...|{'anger': 0, 'ant...|
|WILL YOU TELL ME WHY|will you tell me why|{'anger': 0, 'ant...|
|STRAW HATS ARE OU...|straw hat ar out ...|{'anger': 0, 'ant...|
|THE EASTERN COAST...|the